# Imports

### Libraries

In [38]:
import h5py

import torch
import torchvision
import onnx

from torch import nn
import torch.nn.functional as F

import numpy as np

### Device

In [39]:
torch.cuda.is_available()

True

In [40]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

### Path

In [41]:
path_full_model = "../models/full_model.pt"
path_converted_model = "../models/full_model.onnx"

path_dataset = "../datasets/clean_dataset/dataset.h5py"

### Dataset 

In [42]:
h5f = h5py.File(path_dataset, 'r')

X_train = np.array(h5f['X_train'])
X_test = np.array(h5f['X_test'])
y_train = np.array(h5f['y_train'])
y_test = np.array(h5f['y_test'])

# PyTorch model

### Architecture

The architecture was copied from the previous notebook [fruits_pytorch_model.ipynb](./fruits_pytorch_model.ipynb).

In [43]:
class CNN(nn.Module):

    def __init__(self):
        super(CNN, self).__init__()

        # Features detector
        self.features1 = nn.Sequential(
            
            # Hidden layer 1
            nn.Conv2d(3, 16, kernel_size=3), nn.ReLU(),
            nn.BatchNorm2d(16),

            nn.MaxPool2d((2, 2)),
            nn.Dropout(0.35)
        )

        # Features detector
        self.features2 = nn.Sequential(
            
            # Hidden layer 3
            nn.Conv2d(16, 32, kernel_size=3), nn.ReLU(),
            nn.BatchNorm2d(32),

            nn.MaxPool2d((2, 2)),
            nn.Dropout(0.35)
        )

        # Features detector
        self.features3 = nn.Sequential(
            
            # Hidden layer 3
            nn.Conv2d(32, 64, kernel_size=3), nn.ReLU(),
            nn.BatchNorm2d(64),

            nn.MaxPool2d((2, 2)),
            nn.Dropout(0.35)
        )

        # Classifier
        self.classifier = nn.Sequential(
            
            nn.Linear(64*13*13, 512), nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.50),

            # Output layer
            nn.Linear(512, 108)
        )

    def forward(self, X):

        # Features
        X = self.features1(X)
        X = self.features2(X)
        X = self.features3(X)

        # print(X.shape)

        # Classifier
        X = X.view(-1, 64*13*13)
        #X = X.view(X.size(0), -1)
        X = self.classifier(X)

        return F.log_softmax(X, dim=1)

### Model

In [44]:
model = torch.load(full_model)
model.eval()

CNN(
  (features1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.35, inplace=False)
  )
  (features2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.35, inplace=False)
  )
  (features3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.35, inplace=False)
  )
  (classi

# Convert to ONNX

In [45]:
sample = torch.rand(1, 3, 120, 120).to(device)

In [46]:
torch.onnx.export(
    model,
    sample,
    path_converted_model,
    input_names=["input"],
    output_names=["output"],
    export_params=True)

In [47]:
# Check that the model converted fine
onnx_model = onnx.load(path_converted_model)
onnx.checker.check_model(onnx_model)